In [ ]:
def z_score_normalization(data, features):
    for i in range(len(features)):
        mean_value = data[features[i]].mean()
        median_value = data[features[i]].median()
        std_dev= data [features[i]].std()
        z_scores = (data - mean_value) / std_dev
        return z_scores
    normalized_data = z_score_normalization(raw_data, features)

In [ ]:
def dist_plot(data, feature_list):
    n_cols= 2 
    n_rows = int(np.ceil(len(feature_list)/n_cols)) 
    # Creating figure
    fig = plt.figure(figsize=(16, 4*n_rows))
    outer = GridSpec(n_rows, n_cols, wspace=0.2, hspace=0.3)

    for i in range(len(feature_list)):
        inner = GridSpecFromSubplotSpec(2, 1, subplot_spec=outer[i], 
                                                 wspace=0.1, hspace=0.1, height_ratios=(0.15, 0.85))
        ax_box= plt.Subplot(fig, inner[0])
        sb.boxplot(data=data, x=feature_list[i], color='lightblue', ax=ax_box)
        ax_box.set_xlabel('')
        fig.add_subplot(ax_box)

        mean_value = data[feature_list[i]].mean()
        median_value = data[feature_list[i]].median()
        ax_hist = plt.Subplot(fig, inner[1])
        sb.histplot(data=data, x=feature_list[i], kde=True, ax=ax_hist)
        ax_hist.axvline(mean_value, color='green', linestyle='dotted', linewidth=2, label='Mean')
        ax_hist.axvline(median_value, color='purple', linestyle='dotted', linewidth=2, label='Median')
        ax_hist.legend(loc='lower right', fontsize=10)

        # Calculate skewness and kurtosis
        skewness = data[feature_list[i]].skew()
        kurt = data[feature_list[i]].kurt()
        if skewness < 0:
            x=0.25
        else:
            x=0.95
        # Add skewness and kurtosis as text on the histogram plot
        ax_hist.text(x, 0.85, f"Skewness: {skewness:.2f}\nKurtosis: {kurt:.2f}", 
                         transform=ax_hist.transAxes, verticalalignment='top', horizontalalignment='right',
                         bbox=dict(facecolor='white', edgecolor='gray', boxstyle='round,pad=0.5'),
                    fontsize=10)
        fig.add_subplot(ax_hist)
    plt.tight_layout()
    plt.show(block=False)

In [ ]:
dist_plot(normalized_data, features)

In [ ]:
def log_transformation(data):
    return np.log(data)
data = raw_data
transformed_data = log_transformation(data)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, QuantileTransformer, OneHotEncoder
from sklearn.model_selection import train_test_split
import pandas as pd

def transform_data(df, target, num_features):
    # Ensure num_features is a list
    if isinstance(num_features, pd.Index):
        num_features = num_features.tolist()
        
    # Encoding target
    onehot_encoder = OneHotEncoder(sparse_output=False)
    target_encoded = onehot_encoder.fit_transform(df[[target]])
    
    # Creating a DataFrame with encoded target columns
    target_encoded_df = pd.DataFrame(target_encoded, columns=[f"{target}_{cat}" for cat in onehot_encoder.categories_[0]])
    
    # Concatenate the original DataFrame with the new one-hot encoded target columns
    df = pd.concat([df.drop(target, axis=1), target_encoded_df], axis=1)
    
    # Assigning features and labels
    x = df.drop(target_encoded_df.columns, axis=1)
    y = target_encoded_df
    
    # Splitting the dataset into train and test sets
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=101)
    
    # Standardization and Encoding
    # Define transformers for different column types
    std_scaler = StandardScaler()
    quantile_transformer = QuantileTransformer(output_distribution='normal', random_state=0)

    # Combine transformers for specific columns
    preprocessor = ColumnTransformer([
        ("num", std_scaler, num_features),
        ("num_trns", quantile_transformer, num_features)
    ])
    
    # Fit transformers on training data only
    preprocessor.fit(x_train)

    # Transform train and test data using fitted transformers
    x_train_transformed = preprocessor.transform(x_train)
    x_test_transformed = preprocessor.transform(x_test)
    
    return x_train_transformed, x_test_transformed, y_train, y_test